In [1]:
#importing libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,LabelEncoder
import pickle

In [5]:
#Loading the dataset
data=pd.read_csv('Churn_Modelling.csv')
data.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [6]:
#Preprocessing the data
data=data.drop(['RowNumber','CustomerId','Surname'],axis=1)
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [7]:
#Encoding categorical variables
#Gender
label_encoder_gender=LabelEncoder()
data['Gender']=label_encoder_gender.fit_transform(data['Gender'])
data.head()

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,619,France,0,42,2,0.00,1,1,1,101348.88,1
1,608,Spain,0,41,1,83807.86,1,0,1,112542.58,0
2,502,France,0,42,8,159660.80,3,1,0,113931.57,1
3,699,France,0,39,1,0.00,2,0,0,93826.63,0
4,850,Spain,0,43,2,125510.82,1,1,1,79084.10,0


In [8]:
#Geography-> using onehot encoder
from sklearn.preprocessing import OneHotEncoder
one_hot_encoder_Geo=OneHotEncoder()
geo_encoder=one_hot_encoder_Geo.fit_transform(data[['Geography']])
geo_encoder

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 10000 stored elements and shape (10000, 3)>

In [9]:
#Converting sparse value to array
geo_encoder.toarray()

array([[1., 0., 0.],
       [0., 0., 1.],
       [1., 0., 0.],
       ...,
       [1., 0., 0.],
       [0., 1., 0.],
       [1., 0., 0.]])

In [10]:
#Feature Names of Geography column
one_hot_encoder_Geo.get_feature_names_out(['Geography'])

array(['Geography_France', 'Geography_Germany', 'Geography_Spain'],
      dtype=object)

In [11]:
#converting the encoded data into a DF
geo_encoded_df=pd.DataFrame(geo_encoder.toarray(), columns=one_hot_encoder_Geo.get_feature_names_out(['Geography']))
geo_encoded_df

,Geography_France,Geography_Germany,Geography_Spain
0,1.0,0.0,0.0
1,0.0,0.0,1.0
2,1.0,0.0,0.0
3,1.0,0.0,0.0
4,0.0,0.0,1.0
...,...,...,...
9995,1.0,0.0,0.0
9996,1.0,0.0,0.0
9997,1.0,0.0,0.0
9998,0.0,1.0,0.0


In [12]:
#concatinating the data into the existing df
data=pd.concat([data.drop('Geography', axis=1),geo_encoded_df],axis=1)
data.head()

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited,Geography_France,Geography_Germany,Geography_Spain
0,619,0,42,2,0.00,1,1,1,101348.88,1,1.0,0.0,0.0
1,608,0,41,1,83807.86,1,0,1,112542.58,0,0.0,0.0,1.0
2,502,0,42,8,159660.80,3,1,0,113931.57,1,1.0,0.0,0.0
3,699,0,39,1,0.00,2,0,0,93826.63,0,1.0,0.0,0.0
4,850,0,43,2,125510.82,1,1,1,79084.10,0,0.0,0.0,1.0


In [13]:
#saving the encoders
with open('label_encoder_gender.pkl', 'wb') as file:
    pickle.dump(label_encoder_gender,file)

with open('one_hot_encoder_Geo.pkl', 'wb') as file:
    pickle.dump(one_hot_encoder_Geo, file)

In [14]:
# Dividing into independent and dependent features
X = data.drop('Exited', axis=1)
y = data['Exited']

# Splitting into training and testing data
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=42)

scaler=StandardScaler()
X_train=scaler.fit_transform(X_train)
X_test=scaler.transform(X_test)

In [15]:
#Saving the scaler
with open('scaler.pkl', 'wb') as file:
    pickle.dump(scaler, file)

## ANN Implementation

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.callbacks import EarlyStopping, TensorBoard
import datetime

In [18]:
#building the model 

model = Sequential([
    Dense(64, activation='relu', input_shape=(X_train.shape[1],)), #HL1
    Dense(32, activation='relu'),#HL2
    Dense(1, activation='sigmoid')#O/P layer
])

In [19]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                832       
                                                                 
 dense_1 (Dense)             (None, 32)                2080      
                                                                 
 dense_2 (Dense)             (None, 1)                 33        
                                                                 
Total params: 2945 (11.50 KB)
Trainable params: 2945 (11.50 KB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
#Compilation
opt = tf.keras.optimizers.Adam(learning_rate=0.01)
loss=tf.keras.losses.BinaryCrossentropy()

model.compile(optimizer=opt, loss=loss, metrics=['accuracy'])

In [37]:
#Setting up the TensorBoard
#using the tf.keras.callbacks i.e EarlyStopping and Tensorboard

log_directory="logs/fit/" + datetime.datetime.now().strftime("%Y%m%d-%H%M%S")
tensorflow_callback=TensorBoard(log_dir=log_directory, histogram_freq=1)

In [24]:
#Setting up Early Stopping
early_stopping_callback=EarlyStopping(monitor='val_loss',patience=15, restore_best_weights=True )

In [34]:
#Training the model
history=model.fit(
    X_train,y_train, validation_data=(X_test, y_test), epochs=100,
    callbacks=[tensorflow_callback, early_stopping_callback]
)

Epoch 1/100
250/250 [==============================] - 1s 2ms/step - loss: 0.3194 - accuracy: 0.8689 - val_loss: 0.3459 - val_accuracy: 0.8640
Epoch 2/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3130 - accuracy: 0.8698 - val_loss: 0.3651 - val_accuracy: 0.8635
Epoch 3/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3172 - accuracy: 0.8686 - val_loss: 0.3743 - val_accuracy: 0.8485
Epoch 4/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3114 - accuracy: 0.8696 - val_loss: 0.3638 - val_accuracy: 0.8580
Epoch 5/100
250/250 [==============================] - 0s 1ms/step - loss: 0.3099 - accuracy: 0.8710 - val_loss: 0.3697 - val_accuracy: 0.8520
Epoch 6/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3062 - accuracy: 0.8730 - val_loss: 0.3631 - val_accuracy: 0.8535
Epoch 7/100
250/250 [==============================] - 0s 2ms/step - loss: 0.3084 - accuracy: 0.8712 - val_loss: 0.3651 - val_accuracy: 0.8505

In [27]:
model.save('model.h5')

c:\Users\priya\Coding\Bank_Classf\venv\Lib\site-packages\keras\src\engine\training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [ ]:
#Load Tensorboard Extension
%reload_ext tensorboard
%tensorboard --logdir logs/fit

### Prediction with ANN model

In [4]:
#Loading the model, the encoders and the scalers
from tensorflow.keras.models import load_model

model=load_model('model.h5')

with open('label_encoder_gender.pkl', 'rb') as file:
    label_encoder_gender=pickle.load(file)

with open('one_hot_encoder_Geo.pkl', 'rb') as file:
    one_hot_encoder_Geo=pickle.load(file)

with open('scaler.pkl', 'rb') as file:
    scaler=pickle.load(file)

In [70]:
# Example input data
input_data={
    'CreditScore':800,
    'Geography':'Spain',
    'Gender':'Male',
    'Age':40,
    'Tenure':3,
    'Balance':60000,
    'NumOfProducts':2,
    'HasCrCard':1,
    'IsActiveMember':1,
    'EstimatedSalary':50000
}

In [71]:
#one hot encode geography
geo_encoder=one_hot_encoder_Geo.transform([[input_data['Geography']]]).toarray()
geo_encoded_df=pd.DataFrame(geo_encoder, columns=one_hot_encoder_Geo.get_feature_names_out(['Geography']))
geo_encoded_df

c:\Users\priya\Coding\Bank_Classf\venv\Lib\site-packages\sklearn\utils\validation.py:2749: UserWarning: X does not have valid feature names, but OneHotEncoder was fitted with feature names
  warnings.warn(


,Geography_France,Geography_Germany,Geography_Spain
0,0.0,0.0,1.0


In [72]:
input_df=pd.DataFrame([input_data])
input_df

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,Spain,Male,40,3,60000,2,1,1,50000


In [73]:
input_df['Gender']=label_encoder_gender.transform([input_df['Gender']])
input_df

c:\Users\priya\Coding\Bank_Classf\venv\Lib\site-packages\sklearn\preprocessing\_label.py:129: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, dtype=self.classes_.dtype, warn=True)


,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary
0,800,Spain,1,40,3,60000,2,1,1,50000


In [74]:
input_df=pd.concat([input_df.drop('Geography', axis=1), geo_encoded_df], axis=1)
input_df

,CreditScore,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Geography_France,Geography_Germany,Geography_Spain
0,800,1,40,3,60000,2,1,1,50000,0.0,0.0,1.0


In [75]:
input_scaled=scaler.transform(input_df)
input_scaled

array([[ 1.53956105,  0.91324755,  0.10479359, -0.69539349, -0.25781119,
         0.80843615,  0.64920267,  0.97481699, -0.87683221, -0.99850112,
        -0.57946723,  1.73494238]])

In [76]:
prediction=model.predict(input_scaled)
prediction

1/1 [==============================] - 0s 22ms/step


array([[0.09246507]], dtype=float32)

In [77]:
prediction_probability=prediction[0][0]
prediction_probability

0.09246507

In [78]:
if prediction_probability>0.5:
    print("Customer likely to stay")
else:
    print('Customer Likely to leave')

Customer Likely to leave
